Part 1: Raw mesh --> decimation --> autoproofreading --> save object

In [1]:
%load_ext autoreload
%autoreload 2

import neurd
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz
from pathlib import Path
from neurd.vdi_microns import volume_data_interface as vdi
vdi.set_parameters_for_directory_modules()

/usr/local/lib/python3.8/dist-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [2]:
# load data 
segment_id = 864691135567721964

mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    plot = True
)
mesh

In [4]:
from datasci_tools import pipeline

products = pipeline.PipelineProducts()
products

In [ ]:
# decimate mesh 

decimation_parameters = dict(
    decimation_ratio =0.25,
)

mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ), 
)

print(products)

In [ ]:
# Decomposition + Saving Neuron

from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

Step 3: Autoproofreading

In [ ]:
# autoproofread neuron

# reload neuron
neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

# generate neuron list
neuron_list = neuron_obj_rec.multi_soma_split_execution(
    verbose = False,
)
n1 = neuron_list[0]

In [ ]:
# cell typing to return neuron_obj_axon

from neurd import neuron_pipeline_utils as npu

neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)


In [ ]:

# autoproof
neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)
vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)
